# PyTorch Custom Datasets 

### Import PyTorch setup device agnostic code.

In [ ]:
import torch
from torch import nn

print(torch.__version__)

In [ ]:
device = "cuda" if torch.cuda.is_available() else 'cpu'
device

# Data

dataset is a subset of Food101 dataset


In [ ]:
import requests
import zipfile
from pathlib import Path

data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

if image_path.is_dir():
    print(f"{image_path} directory already exists...skipping download")
else:
    print(f"{image_path} does not exist, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print('downloading pizza, steak, and sushi data')
    f.write(request.content)

with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("unzipping...")
    zip_ref.extractall(image_path)

# Exploring data

In [ ]:
import os
def walk_through_dir(dir_path):
    for dirpath, dirnames, filenames in os.walk(dir_path):
        print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [ ]:
walk_through_dir(image_path)

In [ ]:
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

# Visualizing images

1. get all image path
2. pick random image: random.choice()
3. get the image class name using `pathlib.Path.parent.stem`
4. open image with Pillow
5. show image meta data

In [ ]:
import random
from PIL import Image

# random.seed(42)

image_path_list = list(image_path.glob("*/*/*.jpg"))
image_path_list

random_image_path = random.choice(image_path_list)
random_image_path

image_class = random_image_path.parent.stem
image_class

img = Image.open(random_image_path)

print(f"random_image_path: {random_image_path}")
print(f"Image class: {image_class}")
print(f"Image height: {img.height}")
print(f"Image width: {img.width}")
img

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

img_as_array = np.asarray(img)

plt.figure(figsize=(10,7))
plt.imshow(img_as_array)
plt.title(f"Image class: {image_class} | Image shape: {img_as_array.shape} -> [height, width, color_channels]")
plt.axis(False)

In [ ]:
img_as_array

# Data Transformation

1. data to tensors
2. dataset `torch.utils.data.Dataset` -> DataLoader `torch.utils.data.DataLoader`

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### transform data with `torchvision.transforms`

In [ ]:
# create transorm for an image
data_transform = transforms.Compose([
    # Resize to 64x64
    transforms.Resize(size=(64,64)),
    # Flip the images randomly on horizontal
    transforms.RandomHorizontalFlip(p=0.5),
    # Turn image into torch tensor
    transforms.ToTensor()
])

In [ ]:
data_transform(img).shape

In [ ]:
def plot_transformed_images(image_paths: list, transform, n=3, seed=None):
    """
      Returns random images from an image patch, loads/transforms and plots original vs transformed
    """
    if seed:
      random.seed(seed)
    random_image_paths = random.sample(image_paths, k=n)
    for image_path in random_image_paths:
       with Image.open(image_path) as f:
          fig, ax = plt.subplots(nrows=1, ncols=2)
          ax[0].imshow(f)
          ax[0].set_title(f"Original\nSize:{f.size}")
          ax[0].axis(False)

          # Transform and plot target image
          transformed_image = transform(f).permute(1, 2, 0) # need to change shape for matplotlib
          ax[1].imshow(transformed_image)
          ax[1].set_title(f"Transformed\nSize: {transformed_image.shape}")
          ax[1].axis("off")

          fig.suptitle(f"Class: {image_path.parent.stem}", fontsize=16)

plot_transformed_images(image_paths=image_path_list,
                        transform=data_transform,
                        n=3,
                        seed=42) 

# loading image data from image folder

In [ ]:
from torchvision import datasets
train_data = datasets.ImageFolder(root=train_dir,
                                  transform=data_transform, # a transorm for the data
                                  target_transform=None) # a transform the label/target

test_data = datasets.ImageFolder(root=test_dir,
                                 transform=data_transform)

train_data,test_data

In [ ]:
# class names as list
class_names = train_data.classes
class_names

In [ ]:
# get class names as a dict
class_dict = train_data.class_to_idx
class_dict

In [ ]:
# len of dataset
len(train_data), len(test_data)

In [ ]:
# Index train_data dataset to get single image and label
img, label = train_data[0][0], train_data[0][1]

In [ ]:
print(f"Image Tensor:\n {img}")
print(f"Image shape: {img.shape}")
print(f"Image datatype: {img.dtype}")
print(f"Image label: {label}")
print(f"Label datatype: {type(label)}")

In [ ]:
# Rearrange order of dimensions
img_permute = img.permute(1,2,0)

# print shapes
print(f"Original shape: {img.shape} -> [color_channels, height, width]")
print(f"Image Permute: {img_permute.shape} -> [height, width, color_channel]")

#plt image
plt.figure(figsize=(10,7))
plt.imshow(img_permute)
plt.axis("off")
plt.title(class_names[label], fontsize=14)

# datalaoders

In [ ]:
from torch.utils.data import DataLoader
BATCH_SIZE = 32

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              num_workers=2,
                              shuffle=True)
test_dataloader = DataLoader(dataset=test_data,
                             batch_size=BATCH_SIZE,
                             num_workers=2,
                             shuffle=False)

train_dataloader, test_dataloader

In [ ]:
img, label = next(iter(train_dataloader))

print(f"Image Shape: {img.shape}")
print(f"Label shape: {label.shape}")

# Option 2: Custom dataloader function

1. load images from files
2. get class names from Dataset
3. get clases as dictionary from dataset

Pros:
* can create a `Dataset` out of almost anything
* Not limited to PyTorch pre-built `Dataset` functions

Cons:
* cant create a `Dataset` from almost anything, does not mean it will work
* using a custom `Dataset` often results in writing more code, which is prone to errors or performance issues


All custom dataset in PyTorch often subclass `torch.utils.data.Dataset`

In [ ]:
import os
import pathlib 
import torch

from PIL import Image
from torch.utils.data import Dataset 
from torchvision import transforms
from typing import Tuple, Dict, List

In [ ]:
train_data.classes, train_data.class_to_idx

# Creating a helper function to get class names

1. get the class names using os.scandir()
2. raise an error if class names arent found 
3. turn class names into a dict and list and return them

In [ ]:
target_directory = train_dir
print(f"Target dir: {target_directory}")

class_names_found = sorted([entry.name for entry in list(os.scandir(target_directory))])
class_names_found

In [ ]:
def find_classes(directory: str) -> Tuple[List[str], Dict[str, int]]:
    """ Finds the clas folder names in a target directory"""
    classes = sorted(entry.name for entry in os.scandir(directory) if entry.is_dir())

    if not classes:
        raise FileNotFoundError(f"Couldn't find any classes in {directory}...please check file structure")
    
    class_to_idx = {class_name: i for i, class_name in enumerate(classes)}

    return classes, class_to_idx

In [ ]:
find_classes(target_directory)

# Create custom `dataset` to replicate `ImageFolder`

Creating custom dataset
1. Subclass `torch.utils.data.Dataset`
2. Init our subclas with target directory
3. Create several attributes:
    * paths - paths of images
    * transform - the transform used
    * classes - list of target classes
    * class_to_idx - a dict of target classes mapped to int labels
4. Create function to `load_images()`
5. Overwrite the `__len()__` method to return length of dataset
6. Overwrite the `__getitem()` method to return given sample when passed an index

In [ ]:
# Custom dataset
from torch.utils.data import Dataset 

#1. Subclass 
class ImageFolderCustom(Dataset):
    #2. Init custom dataset
    def __init__(self,
            targ_dir: str,
            transform=None) -> None:
        #3. Create class atrributes
        # gather all image paths
        self.paths = list(pathlib.Path(targ_dir).glob("*/*.jpg"))
        # transform
        self.transform = transform
        # create classes and class_to_idx
        self.classes, self.class_to_idx = find_classes(targ_dir)
    
    # 4. Create a func to load images
    def load_images(self, index: int) -> Image.Image:
        """
        Opens an image via a path and return it.
        """
        image_path = self.paths[index]
        return Image.open(image_path)
    
    #5. overwrite __len__()
    def __len__(self) -> int:
        "return total number of samples."
        return len(self.paths)
    
    #6. overwrite __getitem()__
    def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
        " returns on sample of data, data and label (X, y)"
        img = self.load_images(index)
        class_name = self.paths[index].parent.name # expect path in format: data_folder/class_name/image.jpg
        class_idx = self.class_to_idx[class_name]

        # transform if needed
        if self.transform:
            return self.transform(img), class_idx # return data, label (X, y)
        return img, class_idx # return image and label

In [ ]:
# Create a transform
train_tranforms = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

test_tranforms = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])

In [ ]:
# test imagefoldercustom
train_data_custom = ImageFolderCustom(targ_dir=train_dir,
                                      transform=train_tranforms)

test_data_custom = ImageFolderCustom(targ_dir=test_dir,
                                     transform=test_tranforms)

In [ ]:
train_data_custom, test_data_custom

In [ ]:
len(train_data), len(train_data_custom)

In [ ]:
len(test_data), len(test_data_custom)

In [ ]:
train_data_custom.classes, train_data_custom.class_to_idx

In [ ]:
# check for equality between original and custom
print(train_data_custom.classes==train_data.classes)

In [ ]:
print(test_data_custom.classes==test_data.classes)

## Create a function to display random images

1. take in a `Dataset` and other params
2. cap number of images to 10
3. set random seed
4. list of random samples from the target dataset
5. Setup a matplotlib plot
6. Loop random samples image and plot with matplotlib
7. make sure dims work with matplotlib

In [ ]:
# 1. create func to take in a dataset
def display_random_images(dataset: torch.utils.data.Dataset,
                          classes: List[str] = None,
                          n: int = 10, 
                          display_shape: bool = True,
                          seed: int = None):
    #2.  adjust display if n too high
    if n > 10:
        n = 10
        display_shape = False 
        print(f"For display, purposes, n shouldn't be larget thatn 10, setting display shape to false")
    # 3. Set the seed
    if seed: 
        random.seed(seed)
    # 4. get random samples indexes
    random_samples_idx = random.sample(range(len(dataset)), k=n)

    # 5. setup plot
    plt.figure(figsize=(16,8))

    # 6. loop random sample images
    for i, targ_sample in enumerate(random_samples_idx):
        targ_image, targ_label = dataset[targ_sample][0], dataset[targ_sample][1]

        # 7. correct dims
        targ_image_adjust = targ_image.permute(1,2,0) # [color_channels, h, w] -> [h, w, color_channels]

        # plot
        plt.subplot(1, n, i+1)
        plt.imshow(targ_image_adjust)
        plt.axis("off")
        if classes:
            title = f"Class: {classes[targ_label]}"
            if display_shape:
                title = title + f"\nshape: {targ_image_adjust.shape}"
        plt.title(title)

In [ ]:
# display random images from built in
display_random_images(train_data,
                      n=5,
                      classes=class_names,
                      seed=42)

In [ ]:
# display random images from custom
display_random_images(train_data_custom,n=5,
                      classes=class_names,
                      seed=42)